# Analýza bezpečnostných udalostí
## ...k aktuálnemu stavu riešenia

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

import TemporalClusterer as tc

import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from umap import UMAP

import datetime
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

import seaborn as sns
from preprocess import *

C:\ProgramData\Anaconda3\envs\temporal_event_corr\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
C:\ProgramData\Anaconda3\envs\temporal_event_corr\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
sns.set(style='white', context='notebook', rc={'figure.figsize': (14, 10)})

## Motivace
Detektory na cesnet sieti generujú bezpečnostné události. Predpokladáme že, na sieti dochádza ku koordinovaným útokom a 
analýza časov príchodov udalosti vie odhaliť skupiny útočiacich adries. 

...

Počet aktívnych adries v jednom dni sa drží na úrovni miliónov.    
GRAF z niektorej 

In [5]:
df = pd.read_pickle('../data/2019-08-28.pcl')

tfrom = datetime.datetime.fromisoformat('2019-08-28 00:00:00').timestamp()
tto = datetime.datetime.fromisoformat('2019-08-28 23:59:55').timestamp()

df = df.loc[(df['timestamp'] >= tfrom) & (df['timestamp'] < tto), :]

#c = tc.TemporalClusterer()
#df['labels'] = c.fit_transform(df, [])

## Problém
Data z detektorov sa koncentrujú do textových súborov vo formáte IDEA. Tento nie je vhodný na priamu analýzu a teda data
sa musia predspracovať. 

* ### Preprocessing
Prevod na lepšie strojovo spracovatelný format z tohto: 

{"Category": ["Recon.Scanning"], "Node": [{"Type": ["Relay"], "Name": "cz.cesnet.au1.warden_filer"}, {"SW": ["LaBrea"], "Type": ["Connection", "Tarpit"], "Name": "cz.cesnet.tarpit"}], "EventTime": "2019-07-31T21:44:55Z", "Target": [{"Port": [23], "IP4": ["195.113.255.137"], "Proto": ["tcp"]}], "ConnCount": 1, "Format": "IDEA0", "_CESNET": {"EventTemplate": "labrea-001"}, "CeaseTime": "2019-07-31T21:44:55Z", "Note": "Connections from remote host to never assigned IP", "Source": [{"Port": [53011], "IP4": ["113.70.212.84"], "Proto": ["tcp"]}], "DetectTime": "2019-07-31T21:59:56Z", "ID": "0bee3bce-8328-4368-b529-456a8e171244", "Description": "TCP connections/scan"}

na format zjednodušene reprezentujuci událosť ktorý zachováva len pre dalšie spracovanie dôležité a využitelné informácie,
 pričom nás bude zaujímať najmä IP a časová značka.
 

In [6]:
df.head()

,ip,timestamp,origin,type,line
571,51.89.173.145,1566943209,cz.cesnet.ids_collector.suricata,Availability.DoS,373993
661,78.31.71.111,1566943209,cz.cesnet.collector_nemea.filer,Attempt.Login,438997
724,88.146.250.170,1566943207,cz.muni.ics.csirt.nemea.bruteforce,Attempt.Login,476617
880,147.229.27.169,1566943224,cz.cesnet.ids_collector.suricata,Availability.DoS,592944
1036,195.206.105.217,1566943230,cz.cesnet.collector_nemea.filer,Attempt.Login,698916


* ### Agregácia do časových sérií

Pre účely vyhľadávania skupín s podobným vzorom komunikácie je treba pohľad na komunikácie IP adries v čase. Je treba
zvolit vhodné časové rozlíšenie, aby bol výpočet realizovateľný z hľadiska pamäťového a časového obmedzenia. Na teraz
uchovavame len bool hodnotu pre daný časový slot, teda stráca sa informácie o počte udalostí v danom slote. To bude 
riešené neskor.  

In [10]:
df['quarters'] = np.floor((df.timestamp - tfrom)/(15*60)).astype(np.int)
df = df.loc[(df['quarters'] >= 0),:]# & (df['timestamp'] < tto), :]

dfc = df.groupby('ip')['quarters'].agg([list,'count'])
print(dfc.list.head())
dfc['series'] = dfc.list.apply(get_bin_series, args=[24*4])
dfc.series.head()

ip
1.0.111.127    [10, 19, 23, 28, 32, 34, 36, 37, 41]
1.0.135.241                                [21, 22]
1.0.144.100                                    [80]
1.0.148.171                                    [29]
1.0.148.75                                 [73, 74]
Name: list, dtype: object


ip
1.0.111.127    [False, False, False, False, False, False, Fal...
1.0.135.241    [False, False, False, False, False, False, Fal...
1.0.144.100    [False, False, False, False, False, False, Fal...
1.0.148.171    [False, False, False, False, False, False, Fal...
1.0.148.75     [False, False, False, False, False, False, Fal...
Name: series, dtype: object

* ### Klastrovanie
    Vstupom sú bool vektory, pre každú ip rovnakej dĺžky. V tomto ktoku je výhodne vykonať filtrovanie dát ke zníženie
    náročnosti výpočtu pairwise matice. 
    Výpočet pairwise distance matrice s vhodnou metrikou. O čo sa snžíme je nájst IP adresy ktoré komunikovali v
    rovnakých časových oknách, pričom rozdiely sa budú premietať do vzdialenosti. 
    
    Na toto je vhodná Jaccard metrika: 
    
    1 - len(A union B) / len(A prod B) 
    

Po spočítaní pairwise matice je dalej možné odstrániť nepodobné ip adresy a zjednodušiť tak prácu klastovaciemu algoritmu
Zvolí sa miniálny počet IP adries v klastre, zníži sa pravdepodobnosť náhodnej korelácie     
* ### Verifikacia klastrov
    Ešte nie je vyreišená
    
    Znova presnejšie klastro
## Limitácie

Monitorovacia pipeline zanša do dát artefakty 
    
Je vyriešené ...

## Plány
